In [ ]:
import io
import random
import time

import diffusers
import requests
import torch
import PIL

# To test performance delta.
use_cuda = torch.cuda.is_available()

def inference(prompt, n_images, guidance, steps, seed, img, neg_prompt):
    generator = None
    if use_cuda:
        generator = torch.Generator("cuda").manual_seed(seed) if seed else None
    elif seed:      
        generator = torch.Generator()
        generator.manual_seed(seed)
    pipe = diffusers.StableDiffusionUpscalePipeline.from_pretrained(
        "stabilityai/stable-diffusion-x4-upscaler",
        revision="fp16" if use_cuda else "fp32",
        torch_dtype=torch.float16 if use_cuda else torch.float32)
    # pipe.scheduler = diffusers.DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
    if use_cuda:
        pipe.to("cuda")
        pipe.enable_attention_slicing()
    width, height = img.size
    if width < 128 or height < 128:
        # Calculate the padding needed to make the image dimensions a multiple of 128.
        padding_x = 128 - (width % 128) if width % 128 != 0 else 0
        padding_y = 128 - (height % 128) if height % 128 != 0 else 0
        # Create a white image of the right size to be used as padding.
        padding_img = PIL.Image.new('RGB', (padding_x, padding_y), color=(255, 255, 255, 0))
        # Paste the padding image onto the original image to add the padding.
        img.paste(padding_img, (width, height))
        # Update the image dimensions to include the padding.
        width += padding_x
        height += padding_y
    if width > 128 or height > 128:
        num_tiles_x = int(width / 128)
        num_tiles_y = int(height / 128)
        upscaled_img = PIL.Image.new('RGB', (img.size[0] * 4, img.size[1] * 4))
        for x in range(num_tiles_x):
            for y in range(num_tiles_y):
                print(f"Upscaling tile {x * num_tiles_y + y + 1}/{num_tiles_x * num_tiles_y}")
                tile = img.crop((x * 128, y * 128, (x + 1) * 128, (y + 1) * 128))
                upscaled_tile = pipe(
                    prompt="",
                    image=tile,
                    num_inference_steps=steps,
                    guidance_scale=guidance,
                    # negative_prompt=neg_prompt,
                    generator=generator).images[0]
                upscaled_img.paste(upscaled_tile, (x * upscaled_tile.size[0], y * upscaled_tile.size[1]))
        return [upscaled_img]
    return pipe(
        prompt=prompt,
        image=img,
        num_inference_steps=steps,
        guidance_scale=guidance,
        negative_prompt=neg_prompt,
        generator=generator).images

def run(prompt, image):
    n_images = 1 # number of images
    neg_prompt = ""
    guidance = 7.5 # max = 15
    steps = 25 # [2, 100]
    seed = 10 # random
    print(f"Seed: {seed}")
    start = time.time()
    gallery = inference(prompt, n_images, guidance, steps, seed, image, neg_prompt)
    print("Took %.1fs" % (time.time()-start))
    return gallery[0]

def download(url):
    """Opens an images and returns it."""
    r = requests.get(url)
    img = PIL.Image.open(io.BytesIO(r.content))
    #img = PIL.Image.open(url)
	# Convert indexed (gif) or RGBA (png) into RGA with a white background.
    if img.mode != "RGB":
        if img.mode == "P":
            img = img.convert("RGBA")
        background = PIL.Image.new("RGB", img.size, (255, 255, 255))
        background.paste(img, mask=img.split()[3])
        img = background.convert("RGB")
    img.thumbnail((128, 128))
    return img

img = run("chicken", download("https://a.thumbs.redditmedia.com/d0XveRjSDd3rK6mNuy2Y3HkwDN86yFp2BgETum5KAE4.png"))
img.save("out/chicken1.png")
img